In [ ]:
import pandas as pd

from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import Select
from selenium.common.exceptions import NoSuchElementException
from selenium.webdriver.chrome.options import Options
import time

# Setup and Configuration

## Stuff that you might need to change

In [ ]:
# KM Account, your excel file
excel_file = '.xlsx'
username = ''
password = ''
chromedriver_location = r'chromedriver.exe' #download the chromedriver according to your chrome browse version
url = "https://mentor.kampusmerdeka.kemdikbud.go.id/login"

In [ ]:
# set True for just saving to draft
# set False for Finalize
draft_mode = True

In [ ]:
# Fill Settings
# 0: Fill everyone
# 1: Fill everyone with status in progress (AKA button with "Continue Assessment")
fill_settings = 0
students_fill_list = list() # change this if you want to specifically fill the assessment for certain students

# Execution

## get the data from the excel file

In [ ]:
df = pd.read_excel(excel_file,sheet_name='Sheet1')
df['Student Name Reg'] = df['Student Name Reg'].str.lower()
df_comment = pd.read_excel('ML-36.xlsx',sheet_name='Machine Learning Comment')
topic_list = list(df.columns[4:])


## Login and get to the page for initial assessment

In [ ]:
# Initialize the selenium driver and open the page
options = Options()
#options.add_argument("--headless=new")
#options.add_argument("--window-size=1920,1080")
driver = webdriver.Chrome(options=options,service=Service(chromedriver_location))
driver.get(url)

In [ ]:
# Login your stuff
driver.find_element(By.XPATH, '//input[@type="email"]').send_keys(username)
driver.find_element(By.XPATH, '//input[@type="password"]').send_keys(password)
driver.find_element(By.XPATH, '//p[text()="Log In"]').click()

In [ ]:
# Get to the initial assessment
time.sleep(2)
driver.refresh()
time.sleep(2)
driver.find_element(By.XPATH, '//p[text()="Batch 4 - Studi Independen"]').click()
time.sleep(2)
driver.find_element(By.XPATH, '//p[text()="Evaluation"]').click()
time.sleep(2)

## Double check the name

In [ ]:
# apparently some of the name doesn't match
# mine has some double spaces and typos
name_correct = True
for student in df['Student Name Reg']:
    try:
        #a little bit of rant
        #some of the is full uppercase
        #some is only capitalized
        xpath = '//p[  contains(translate(., "ABCDEFGHIJKLMNOPQRSTUVWXYZ", "abcdefghijklmnopqrstuvwxyz"), "{}")]'.format(student)
        child_elem_mainmenu = driver.find_element(By.XPATH, xpath)
    except:
        name_correct = False
        print("student {} doesn't exist in Kampus Merdeka page",student)


## Iterate over the students

In [ ]:
if name_correct:
    for student in df['Student Name Reg']:
        # Find the fucking assessment button for the student
        # fuck sake i feel like there is a better way to do this
        # god knows how many hours it took to figure this out
        xpath = '//p[contains(translate(., "ABCDEFGHIJKLMNOPQRSTUVWXYZ", "abcdefghijklmnopqrstuvwxyz"), "{}")]'.format(student)
        child_elem_mainmenu = driver.find_element(By.XPATH, xpath)
        parent_elem_mainmenu = child_elem_mainmenu.find_element(By.XPATH, "..").find_element(By.XPATH, "..")
        parent_cubic = parent_elem_mainmenu.find_element(By.XPATH, "..")
        bunch_of_childs = parent_cubic.find_elements(By.XPATH, "./*")[4]
        shitty_grandchild = bunch_of_childs.find_element(By.XPATH, "./*")
        thefuckinggoddamnshittybutton = shitty_grandchild.find_element(By.XPATH, "./div")
        button_text = thefuckinggoddamnshittybutton.text

        # Check condition
        if fill_settings == 0:
            do_fill = True
        elif fill_settings == 1 and button_text == "Assess":
            do_fill = True
        elif fill_settings == 2 and button_text == "Continue Assessment":
            do_fill = True
        elif fill_settings == 3 and student in students_fill_list:
            do_fill = True
        else:
            do_fill = False

        if do_fill:
            thefuckinggoddamnshittybutton.click()
            time.sleep(1)
            # Get the student name and the data from df
            student_name = driver.find_element(By.XPATH, "//p[@class='text-0-0-26 sans-0-0-40 heading-1-0-0-51 name-0-0-450']").text.lower()
            current_row = df.loc[df['Student Name Reg'] == student_name]

            # Fill all the score and comment
            for topic in topic_list:
                child_elem = driver.find_element(By.XPATH, '//p[normalize-space(text())="{}"]'.format(topic))
                parent_elem = child_elem.find_element(By.XPATH, "..")
                textarea_elem = parent_elem.find_element(By.TAG_NAME, "textarea")

                # Get the score and the comment
                score = current_row[topic].values[0]
                # I hardcoded this one because honestly not worth automating



                if score >= 70:
                    comment = df_comment.loc[df_comment['Course List'] == topic][df_comment.columns[1]]
                elif score >= 40:
                    comment = df_comment.loc[df_comment['Course List'] == topic][df_comment.columns[2]]
                else: 
                    comment = df_comment.loc[df_comment['Course List'] == topic][df_comment.columns[3]]

                # fill the comment
                text_area = parent_elem.find_element(By.TAG_NAME, "textarea")
                # Fucking hell, why tf text_area.clear() doesn't work as intended
                # Alas, ctrl+a & del is the way
                text_area.send_keys(Keys.CONTROL + "a")
                text_area.send_keys(Keys.DELETE)
                text_area.send_keys(comment.values[0])

                # fill the score
                dropdown_elem = parent_elem.find_element(By.TAG_NAME, "select")
                Select(dropdown_elem).select_by_value(str(score))

            # Doesn't work otherwise
            # Save the draft or Finalize and automatically go to the evaluation page again
            driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
            if draft_mode:
                retry_count = 0
                while retry_count <= 10:
                    try:
                        driver.find_element(By.XPATH, '//p[text()="Save as Draft"]').click()
                    except:
                        time.sleep(1)
                        retry_count += 1
                        continue
                    break
                retry_count = 0
                while retry_count <= 10:
                    try:
                        driver.find_element(By.XPATH, '//p[text()="OK"]').click() # This sends you back automatically
                    except:
                        time.sleep(1)
                        retry_count += 1
                        continue
                    break
            
            else:
                pass # I haven't tried finalize hahaha
            print("student {} evaluation data has been filled".format(student))
            time.sleep(5)

